In [1]:
!pwd

/content


In [2]:
import pandas as pd
import re


In [ ]:
import pandas as pd
df_re=pd.read_csv('file_2_USCSre.csv')
df_re_filter=df_re[['Borehole ID', 'Description', 'USCS', 'Lithology', 'USCS_predict', 'USCS_same']]
df_re_filter.columns=['Borehole ID', 'Description', 'USCS', 'Lithology', 'USCS Predicted by Lithology', 'USCS Prediction Match']
df_re_filter[df_re_filter['Borehole ID']==2]

,Borehole ID,Description,USCS,Lithology,USCS Predicted by Lithology,USCS Prediction Match
3,2,"Duff, topsoil and roots, moist",PT,Topsoil / vegetation,PT,True
4,2,"Loose, reddish-brown, silty sand, moist to wet",SM,Silty sand,SM,True
5,2,"Medium dense to dense, tan-brown, mottled, sil...",SM,Silty sand,SM,True
6,2,"Very stiff, gray-brown, mottled, very fine san...",ML,Sandy silt,SW-SP,False


## Original Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
boreholes = pd.read_csv("drive/MyDrive/file_1.csv")
boreholes = boreholes.iloc[:, [0, 3, 4, 5]]
boreholes.columns = ["Borehole ID", "Latitude", "Longitude", "Elevation"]
boreholes

,Borehole ID,Latitude,Longitude,Elevation
0,1,47.574251,-122.407051,148.80
1,2,47.574177,-122.407057,149.50
2,3,47.574220,-122.407281,142.20
3,4,47.574951,-122.406318,129.80
4,5,47.574938,-122.406120,136.20
...,...,...,...,...
100405,153207,47.436977,-122.242980,26.33
100406,153208,47.437219,-122.242422,25.45
100407,153209,47.437408,-122.242929,22.69
100408,153210,47.437303,-122.243141,24.02


In [5]:
layering = pd.read_csv("drive/MyDrive/file_2.csv").drop(columns=["Document ID", "Latitude", "Longitude"])
layering

,Borehole ID,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology
0,1,1,0.0,1.2,"Duff and roots over loose, dark brown, silty f...",PT,Fill
1,1,2,1.2,2.7,"Soft, gray-brown, mottled, very fine, sandy si...",ML,Sandy silt
2,1,3,2.7,8.0,"Very stiff to hard, gray-brown, silt with some...",ML,Silt
3,2,1,0.0,0.8,"Duff, topsoil and roots, moist",PT,Topsoil / vegetation
4,2,2,0.8,2.5,"Loose, reddish-brown, silty sand, moist to wet",SM,Silty sand
...,...,...,...,...,...,...,...
341153,153209,3,25.0,28.5,"Clean silty sands, medium to dense to dense.",SM-SP,Silty sand
341154,153210,1,0.0,6.0,"Silty to clayey sands and sandy silts, loose.",SM-ML,Clayey sand
341155,153210,2,6.0,19.0,"Fine sandy and clayey silts, very soft to medi...",ML,Clayey silt
341156,153210,3,19.0,30.0,"Silty sand, medium dense to dense.",SM-SP,Silty sand


### with USCS_predict version

In [ ]:
layering_pred=pd.read_csv("file_2_USCSre.csv").drop(columns=["USCS_prob"])
layering_pred.columns=['Borehole ID', 'Sub Borehole Layer', 'Top Depth', 'Bottom Depth',
       'Description', 'USCS', 'Lithology', 'USCS Predicted by Lithology', 'USCS Prediction Match']
layering_pred

In [ ]:
layering=layering_pred

In [6]:
layering.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341158 entries, 0 to 341157
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Borehole ID         341158 non-null  int64  
 1   Sub Borehole Layer  341158 non-null  int64  
 2   Top Depth           341158 non-null  float64
 3   Bottom Depth        341158 non-null  float64
 4   Description         341158 non-null  object 
 5   USCS                174864 non-null  object 
 6   Lithology           341158 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 18.2+ MB


In [7]:
print("Number of boreholes with layering data:", layering["Borehole ID"].nunique())

Number of boreholes with layering data: 87574


In [8]:
layering["USCS"].unique()

array(['PT', 'ML', 'SM', 'Not applicable', 'GC', 'CL', 'CH', 'OH', 'SC',
       'CL-ML', 'SC-SM', 'OL', 'ML-CL', 'SM-ML', 'SW', 'SP', 'SP-ML',
       'ML-SP', nan, 'GM', 'MH', 'SP-SM', 'ML-SM', 'SM-SP', 'GP', 'SW-SM',
       'Not applicable-ML', 'Not applicable-OL', 'GW', 'SM-SW', 'OL-SM',
       'GP-GM', 'GW-GM', 'OL - SM', 'SP-SW', 'SW-GW', 'GM-ML',
       'Non-standard', 'GW-SW', 'ML-SW', 'CL-CH', 'ML-OL', 'SP-GP',
       'SM-OL', 'SM-GM', 'ML-CH', 'SM-GP', 'GM-SP', 'CL-GM', 'SM-CL',
       'CL-SM', 'CL-SC', 'ML-SC', 'SP-CL', 'PT-ML', 'ML-PT', 'GL',
       'ML-MH', 'OH-CL', 'ML-GM', 'GM-SM', 'MH-OH', 'GP-SP', 'SW-GM',
       'SP-GM', 'SW-ML', 'OL-ML', 'SP-PT', 'PT-OL', 'Now applicable',
       'Not applicabble', 'SM/ML', 'CH-ML', 'Not appliable', 'SW-PT',
       'Not application', 'CL-SP', 'SP-SC', 'SW-SC', 'GW-GC', 'GC-GM',
       'GP-GC', 'SW-SP', 'GM-GP', 'CL-SM-CL', 'CL-SW-CL', 'CL-SW-PT-CL',
       'CL-SM-PT', 'SC-CL', 'SM-SC', 'CH-CL', 'OL-PT', 'CH-MH', 'OH-PT',
       'SM-CH'

In [9]:
sptn = pd.read_csv("drive/MyDrive/file_3.csv").drop(columns=["Document ID", "Latitude", "Longitude"])
sptn

,Borehole ID,Borehole Test ID,Test Depth,SPTN
0,7,1,33.0,60.0
1,7,2,35.5,58.0
2,7,3,38.0,80.0
3,7,4,40.5,84.0
4,8,1,33.5,94.0
...,...,...,...,...
29990,153205,9,32.5,44.0
29991,153205,10,37.5,23.0
29992,153205,11,42.5,43.0
29993,153205,12,47.5,27.0


In [10]:
sptn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29995 entries, 0 to 29994
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Borehole ID       29995 non-null  int64  
 1   Borehole Test ID  29995 non-null  int64  
 2   Test Depth        29995 non-null  float64
 3   SPTN              23026 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 937.5 KB


In [11]:
print("Number of boreholes with SPT data:", sptn["Borehole ID"].nunique())

Number of boreholes with SPT data: 4065


### Status Record

In [12]:
boreholeid_list=list(set(layering['Borehole ID'].tolist())&set(boreholes['Borehole ID'].tolist())&set(sptn['Borehole ID'].tolist()))
boreholeid_list.sort()
borehole_status=pd.DataFrame({'Borehole ID':boreholeid_list, "1+SPTN per layer":False,"Evenly spread":False,"Start from top": False, 'Start from bottom': False, 'Start from either': False})
borehole_status
borehole_status

,Borehole ID,1+SPTN per layer,Evenly spread,Start from top,Start from bottom,Start from either
0,7,False,False,False,False,False
1,8,False,False,False,False,False
2,15,False,False,False,False,False
3,16,False,False,False,False,False
4,18,False,False,False,False,False
...,...,...,...,...,...,...
3927,153201,False,False,False,False,False
3928,153202,False,False,False,False,False
3929,153203,False,False,False,False,False
3930,153204,False,False,False,False,False


## Data Preprocessing

### Merge all the data -> `boreholes_uscs_sptn`

In [13]:
uscs_sptn = pd.merge(sptn, layering, on=["Borehole ID"], how="inner")
print("Number of boreholes with both USCS and SPTN data:", uscs_sptn["Borehole ID"].nunique())

Number of boreholes with both USCS and SPTN data: 3932


In [14]:
# If the test depth is right at the junction of two layers, it is assigned to the lower layer.
uscs_sptn = uscs_sptn[(uscs_sptn["Test Depth"] >= uscs_sptn["Top Depth"]) &
                      (uscs_sptn["Test Depth"] < uscs_sptn["Bottom Depth"])]
# If the test depth is right at the junction of two layers, it is assigned to both layers.
# uscs_sptn = uscs_sptn[(uscs_sptn["Test Depth"] >= uscs_sptn["Top Depth"]) &
#                       (uscs_sptn["Test Depth"] <= uscs_sptn["Bottom Depth"])]
uscs_sptn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28932 entries, 7 to 186981
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Borehole ID         28932 non-null  int64  
 1   Borehole Test ID    28932 non-null  int64  
 2   Test Depth          28932 non-null  float64
 3   SPTN                22223 non-null  float64
 4   Sub Borehole Layer  28932 non-null  int64  
 5   Top Depth           28932 non-null  float64
 6   Bottom Depth        28932 non-null  float64
 7   Description         28932 non-null  object 
 8   USCS                18483 non-null  object 
 9   Lithology           28932 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 2.4+ MB


In [15]:
boreholes_uscs_sptn = pd.merge(boreholes, uscs_sptn, on=["Borehole ID"], how="right")
boreholes_uscs_sptn

,Borehole ID,Latitude,Longitude,Elevation,Borehole Test ID,Test Depth,SPTN,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology
0,7,47.575005,-122.406037,131.30,1,33.0,60.0,8,11.5,36.5,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay
1,7,47.575005,-122.406037,131.30,2,35.5,58.0,8,11.5,36.5,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay
2,7,47.575005,-122.406037,131.30,3,38.0,80.0,11,38.0,41.0,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand
3,7,47.575005,-122.406037,131.30,4,40.5,84.0,11,38.0,41.0,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand
4,8,47.574982,-122.406239,130.90,1,33.5,94.0,6,31.5,36.0,"2 inches of moist, gray/green, clayey silty fi...",ML,Silty sand
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28927,153205,47.437412,-122.242456,23.75,9,32.5,44.0,6,27.0,54.0,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand
28928,153205,47.437412,-122.242456,23.75,10,37.5,23.0,6,27.0,54.0,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand
28929,153205,47.437412,-122.242456,23.75,11,42.5,43.0,6,27.0,54.0,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand
28930,153205,47.437412,-122.242456,23.75,12,47.5,27.0,6,27.0,54.0,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand


In [16]:
boreholes_uscs_sptn.describe()

,Borehole ID,Latitude,Longitude,Elevation,Borehole Test ID,Test Depth,SPTN,Sub Borehole Layer,Top Depth,Bottom Depth
count,28932.000000,27675.000000,27675.000000,27675.000000,28932.000000,28932.000000,22223.000000,28932.000000,28932.000000,28932.000000
mean,101700.642057,47.552394,-122.233569,194.004630,6.476808,27.405902,31.547451,4.195597,20.436780,35.212235
std,60849.159339,0.289547,0.835045,365.816722,6.101861,26.798495,32.834224,2.995071,24.388595,29.704603
min,7.000000,46.136920,-123.347181,1.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.080000
25%,2501.000000,47.465250,-122.405565,22.300000,2.000000,8.000000,9.000000,2.000000,3.000000,14.000000
50%,135101.000000,47.584469,-122.334081,96.800000,5.000000,19.500000,22.000000,3.000000,12.000000,27.000000
75%,142979.000000,47.624146,-122.258794,219.500000,9.000000,37.500000,44.000000,5.000000,29.530001,47.700001
max,153205.000000,48.971782,-117.079555,2510.000000,73.000000,290.000000,1170.000000,34.000000,275.000000,298.000000


### Generate new columns & Rename columns

In [17]:
# Convert depth to elevation
boreholes_uscs_sptn["Top Depth"] = boreholes_uscs_sptn["Elevation"] - boreholes_uscs_sptn["Top Depth"]
boreholes_uscs_sptn["Bottom Depth"] = boreholes_uscs_sptn["Elevation"] - boreholes_uscs_sptn["Bottom Depth"]
boreholes_uscs_sptn["Test Depth"] = boreholes_uscs_sptn["Elevation"] - boreholes_uscs_sptn["Test Depth"]

In [18]:
# Add a column for density/consistency
def get_density(uscs, sptn):
    if type(uscs) != str:
        return None
    types = re.findall(r'\b[A-Z]{2}\b', uscs)
    coarse, fine= False, False
    for t in types:
        if t in ["GW", "GP", "GM", "GC", "SW", "SP", "SM", "SC"]:
            coarse = True
            break
        elif t in ["ML", "CL", "OL", "MH", "CH", "OH", "PT"]:
            fine = True

    if coarse:
        if sptn <= 4:
            return 'Very loose'
        elif sptn <= 10:
            return 'Loose'
        elif sptn <= 24:
            return 'Medium'
        elif sptn <= 50:
            return 'Dense'
        else:
            return 'Very Dense'
    elif fine:
        if sptn <= 2:
            return 'Very soft'
        elif sptn <= 4:
            return 'Soft'
        elif sptn <= 8:
            return 'Medium'
        elif sptn <= 15:
            return 'Stiff'
        elif sptn <= 30:
            return 'Very stiff'
        else:
            return 'Hard'

boreholes_uscs_sptn['Density/Consistency'] = boreholes_uscs_sptn.apply(
    lambda row: get_density(row['USCS'], row['SPTN']), axis=1)

In [19]:
boreholes_uscs_sptn.columns

Index(['Borehole ID', 'Latitude', 'Longitude', 'Elevation', 'Borehole Test ID',
       'Test Depth', 'SPTN', 'Sub Borehole Layer', 'Top Depth', 'Bottom Depth',
       'Description', 'USCS', 'Lithology', 'Density/Consistency'],
      dtype='object')

In [20]:
# Rename the columns
boreholes_uscs_sptn.columns = ["Borehole ID", "Latitude", "Longitude", "Elevation (ft)",
                               "In-situ Test #", "Test Elevation (ft)", "SPTN",
                               "Layer #", "Layer Top Elevation (ft)", "Layer Bottom Elevation (ft)",
                               "Soil Description", "USCS", "Lithology",'USCS Predicted by Lithology',
                              'USCS Prediction Match', "Density / Consistency"]

ValueError: Length mismatch: Expected axis has 14 elements, new values have 16 elements

In [21]:
boreholes_uscs_sptn[boreholes_uscs_sptn["Borehole ID"] == 21]

,Borehole ID,Latitude,Longitude,Elevation,Borehole Test ID,Test Depth,SPTN,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology,Density/Consistency
9,21,47.568436,-122.40748,77.3,1,77.3,3.0,1,77.3,75.8,"Grayish brown, dry, very loose, well graded sa...",SW,Fill,Very loose
10,21,47.568436,-122.40748,77.3,2,75.8,12.0,2,75.8,70.3,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Stiff
11,21,47.568436,-122.40748,77.3,3,74.3,8.0,2,75.8,70.3,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium
12,21,47.568436,-122.40748,77.3,4,73.3,7.0,2,75.8,70.3,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium
13,21,47.568436,-122.40748,77.3,5,71.8,7.0,2,75.8,70.3,"Clay, medium gray, stiff, slightly moist, with...",CL,Clay,Medium
14,21,47.568436,-122.40748,77.3,6,70.3,6.0,3,70.3,68.8,"Sandy silt, brown, medium stiff, wet, over san...",ML-CL,Sandy silt,Medium
15,21,47.568436,-122.40748,77.3,7,68.8,5.0,4,68.8,67.8,"Sand, rust brown to brown, loose, wet, with mi...",SP,Sand,Loose
16,21,47.568436,-122.40748,77.3,8,67.8,6.0,5,67.8,66.8,"Sand, rust brown, loose, moist, some oxidation...",SP-ML,Sand,Loose
17,21,47.568436,-122.40748,77.3,9,66.8,16.0,6,66.8,65.3,"Sandy silt, mottled gray and rusty brown, very...",ML-SP,Sandy silt,Medium
18,21,47.568436,-122.40748,77.3,10,65.3,37.0,7,65.3,63.8,"Sand, brown, dense, moist to wet, well graded ...",SW,Sand,Dense


## Missing SPT-N Values

### Status: at least 1 SPTN per layer

In [ ]:
uscs_left2_sptn=layering.merge(sptn, on='Borehole ID', how='left')
uscs_left2_sptn['In Range'] = (uscs_left2_sptn['Test Depth'] >=uscs_left2_sptn['Top Depth']) & (uscs_left2_sptn['Test Depth'] < uscs_left2_sptn['Bottom Depth'])
uscs_left2_sptn['In Range_int']=uscs_left2_sptn['In Range'].astype(int)
uscs_left2_sptn

In [ ]:

uscs_left2_sptn_count=uscs_left2_sptn.groupby(['Borehole ID', 'Sub Borehole Layer'])['In Range_int'].sum().reset_index()
uscs_left2_sptn_count


In [ ]:
zero_count_per_borehole = uscs_left2_sptn_count.groupby('Borehole ID')['In Range_int'].apply(lambda x: (x == 0).sum())
zero_count_per_borehole.reset_index()
valid_id=zero_count_per_borehole[zero_count_per_borehole==0].index.tolist()
zero_count_per_borehole[zero_count_per_borehole==0]


In [ ]:
# # # Filter Borehole IDs where at most 1 layer has no test
# # tolerance_level=1
# # result_boreholes_list = zero_count_per_borehole[zero_count_per_borehole <= tolerance_level].index.tolist()
# # result_boreholes_list

# # Count number of sublayers per Borehole ID
# layer_count = uscs_left2_sptn_count.groupby('Borehole ID')['Sub Borehole Layer'].count()
# print(layer_count)

In [ ]:
# # Count number of missing tests (In Range_int == 0) per Borehole ID
# zero_count = uscs_left2_sptn_count.groupby('Borehole ID')['In Range_int'].apply(lambda x: (x == 0).sum())
# zero_count.reset_index()
# zero_count[zero_count==0]

In [ ]:


# Apply filtering logic:
# - Single-layer boreholes must have at least one test (zero_count == 0)
# - Multi-layer boreholes can have at most one missing test (zero_count <= 1)

# valid_boreholes = zero_count[zero_count==0].index.tolist()
# # valid_boreholes = zero_count[(layer_count > 1) & (zero_count <= 1)].index.tolist()


uscs_left2_sptn[uscs_left2_sptn['Borehole ID'].isin(valid_id) & uscs_left2_sptn['In Range']==True]

#### update

In [ ]:
borehole_status.loc[borehole_status['Borehole ID'].isin(valid_id), '1+SPTN per layer'] = True
borehole_status

### Status: start from
Layering Depth Range vs. SPTN Depth Range

In [ ]:
import numpy as np
import pandas as pd

layering=pd.read_csv("file_2.csv")
layering

In [ ]:
tolerance_diff=10.1
uscs_range = layering.groupby("Borehole ID").agg(
    Total_Top_Depth=("Top Depth", "min"),
    Total_Bottom_Depth=("Bottom Depth", "max")
)
uscs_range

In [ ]:
sptn_range = sptn.groupby("Borehole ID")['Test Depth'].agg(['min', 'max'])
sptn_range

In [ ]:
range = pd.merge(uscs_range, sptn_range, on=["Borehole ID"], how="inner")
range

In [ ]:
range[(abs(range["Total_Top_Depth"] - range["min"]) < tolerance_diff)]

#### update

In [ ]:
id_starttop=range[(abs(range["Total_Top_Depth"] - range["min"]) < tolerance_diff)].index
id_startbottom=range[(abs(range["Total_Bottom_Depth"] - range["max"]) < tolerance_diff)].index
id_startany=range[(abs(range["Total_Top_Depth"] - range["min"]) < tolerance_diff) | (abs(range["Total_Bottom_Depth"] - range["max"]) < tolerance_diff)].index
borehole_status.loc[borehole_status['Borehole ID'].isin(id_starttop), 'Start from top'] = True
borehole_status.loc[borehole_status['Borehole ID'].isin(id_startbottom), 'Start from bottom'] = True
borehole_status.loc[borehole_status['Borehole ID'].isin(id_startany), 'Start from either'] = True
borehole_status

### Status: spread evenly

In [ ]:
import numpy as np
def is_valid_depth_group(depths):
    depths_sorted = depths.sort_values().values  # Sort depths and convert to numpy array
    # Calculate consecutive differences using numpy's diff function
    differences = np.diff(depths_sorted)
    return np.all(differences < tolerance_diff)  # Check if all differences are less than 6

# Apply the function, ensuring we work with 'Test Depth' only after grouping by 'Borehole ID'
even_df = sptn.groupby('Borehole ID')['Test Depth'].apply(lambda depths: is_valid_depth_group(depths))

# Get the IDs where the condition is met (True values)
even_id = even_df[even_df].index.tolist()

borehole_status.loc[borehole_status['Borehole ID'].isin(even_id), 'Evenly spread'] = True
borehole_status


,Borehole ID,1+SPTN per layer,Evenly spread,Start from top,Start from bottom,Start from either
0,7,False,True,False,True,True
1,8,False,False,False,True,True
2,15,False,True,False,True,True
3,16,False,True,False,True,True
4,18,False,True,False,True,True
...,...,...,...,...,...,...
3927,153201,True,True,True,True,True
3928,153202,True,True,True,True,True
3929,153203,True,True,True,True,True
3930,153204,False,True,True,True,True


In [ ]:
print(borehole_status['1+SPTN per layer'].value_counts())
print(borehole_status['Start from top'].value_counts())
print(borehole_status['Start from bottom'].value_counts())
print(borehole_status['Evenly spread'].value_counts())
print(borehole_status['Start from either'].value_counts())

1+SPTN per layer
False    2875
True     1057
Name: count, dtype: int64
Start from top
True     3163
False     769
Name: count, dtype: int64
Start from bottom
True     3580
False     352
Name: count, dtype: int64
Evenly spread
True     3146
False     786
Name: count, dtype: int64
Start from either
True     3815
False     117
Name: count, dtype: int64


In [ ]:
print('even & start from somewhere: ', len(borehole_status[(borehole_status['Evenly spread']==True) & (borehole_status['Start from either']==True)]))
print('even & start from both: ', len(borehole_status[(borehole_status['Evenly spread']==True) & (borehole_status['Start from bottom']==True) & (borehole_status['Start from top']==True)]))
print('even & start from top: ', len(borehole_status[(borehole_status['Evenly spread']==True) & (borehole_status['Start from top']==True)]))

even & start from somewhere:  3068
even & start from both:  2472
even & start from top:  2660


In [ ]:
# print(borehole_status.columns)

In [ ]:
# from upsetplot import UpSet, from_indicators
# import matplotlib.pyplot as plt

# # Convert boolean columns to UpSet format
# upset_data = from_indicators(borehole_status.columns[1:], borehole_status)

# # Create and show the UpSet plot
# UpSet(upset_data).plot()
# plt.show()


### export

In [ ]:
borehole_status.to_csv('borehole_status.csv', index=False)

In [ ]:
borehole_withstatus=boreholes.merge(borehole_status, on='Borehole ID', how='inner')
borehole_withstatus.to_csv('borehole_withstatus_level'+str(tolerance_diff)+'.csv', index=False)

### (skip, darft) Whether SPTN test depth is even

In [ ]:
def is_evenly_distributed(lst, tolerance=1):
    # Remove NaN values
    lst = [x for x in lst if pd.notna(x)]

    if len(lst) < 3:
        return False  # Too few valid points to judge

    # Calculate differences between consecutive numbers
    diffs = [lst[i+1] - lst[i] for i in range(len(lst)-1)]
    avg_diff = sum(diffs) / len(diffs)

    if avg_diff == 0:  # Prevent division by zero
        return False  # All values are identical, so it's evenly distributed

    # Check if the variation in differences is within tolerance
    deviations = [abs(d - avg_diff) / avg_diff for d in diffs]

    return max(deviations) <= tolerance  # If max deviation is small, it's evenly distributed


In [ ]:
uscs_sptn_all_even=uscs_sptn_all.copy().dropna()
uscs_sptn_all_even['even'] = uscs_sptn_all_even.groupby('Borehole ID')['Test Depth'].transform(lambda x: is_evenly_distributed(sorted(x.tolist())))


NameError: name 'uscs_sptn_all' is not defined

In [ ]:
uscs_sptn_all_even_filter=uscs_sptn_all_even[uscs_sptn_all_even['even']==True]
print(uscs_sptn_all_even_filter['Borehole ID'].nunique())
uscs_sptn_all_even_filter

NameError: name 'uscs_sptn_all_even' is not defined

## Lithology => USCS (too slow, can skip)

In [ ]:
dict_uscsRules_tokens_string={
    "GW": "Coarse-Grained Soils, Gravels, Clean Gravels, Well-graded gravels, gravel-sand mixtures, little no fines",
    "GP": "Coarse-Grained Soils, Gravels, Clean Gravels, Poorly-graded gravels, gravel-sand mixtures, little no fines",
    "GM": "Coarse-Grained Soils, Gravels, Gravels with Fines, Silty gravels, gravel-sand-silt mixtures",
    "GC": "Coarse-Grained Soils, Gravels, Gravels with Fines, Clayey gravels, gravel-sand-clay mixtures",
    "SW": "Coarse-Grained Soils, Sands, Clean Sands, Well-graded sands, gravelly sands, little no fines",
    "SP": "Coarse-Grained Soils, Sands, Clean Sands, Poorly-graded sands, gravelly sands, little no fines",
    "SM": "Coarse-Grained Soils, Sands, Sands with Fines, Silty sands, sand-silt mixtures",
    "SC": "Coarse-Grained Soils, Sands, Sands with Fines, Clayey sands, sand-clay mixtures",
    "ML": "Fine-Grained Soils, Silts Clays, Inorganic silts, very fine sands, rock flour, silty clayey fine sands, clayey silts slight plasticity",
    "CL": "Fine-Grained Soils, Silts Clays, Inorganic clays low medium plasticity, gravelly clays, sandy clays, silty clays, lean clays",
    "OL": "Fine-Grained Soils, Silts Clays, Organic silts, organic silty clays low plasticity",
    "MH": "Fine-Grained Soils, Silts Clays, Inorganic silts, micaceous diatomaceous fine sandy silty soils, elastic silts",
    "CH": "Fine-Grained Soils, Silts Clays, Inorganic clays high plasticity, fat clays",
    "OH": "Fine-Grained Soils, Silts Clays, Organic clays medium high plasticity, organic silts",
    "PT": "Fine-Grained Soils, Highly Organic Soils, Peat other highly organic soils"
}


In [ ]:
import pandas as pd
import nltk
from collections import Counter
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt_tab')

# Make sure to download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Preprocess function for tokenizing, stemming, stopword removal, and cleaning
def preprocess_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize each token
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

# Apply preprocessing to all descriptions in dict_uscsRules_tokens_string
dict_uscsRules_tokens_clean = {}
for label, description in dict_uscsRules_tokens_string.items():
    dict_uscsRules_tokens_clean[label] = preprocess_text(description)

# Define the function to create the prob list and prediction label
def create_prob_list(row, threshold_1=0.1, threshold_2=0.05):
    # Generate the label-probability pairs
    label_prob_pairs = []
    for label, tokens in dict_uscsRules_tokens_clean.items():
        prob = 0
        for token in tokens:
            # Check how many tokens match between the row's description and the USCS description
            if token in row['Description'].lower():  # Use lowercased matching
                prob += 1
        # Normalize probability (for example, by the number of tokens in the description)
        prob = prob / len(tokens) if len(tokens) > 0 else 0
        label_prob_pairs.append((label, prob))

    # Sort by probability in descending order
    sorted_label_prob_pairs = sorted(label_prob_pairs, key=lambda x: x[1], reverse=True)

    # Extract the top labels and probabilities
    top_label = sorted_label_prob_pairs[0]
    second_label = sorted_label_prob_pairs[1] if len(sorted_label_prob_pairs) > 1 else None
    third_label = sorted_label_prob_pairs[2] if len(sorted_label_prob_pairs) > 2 else None

    # Calculate the difference between the top label and second label probabilities (if available)
    top_diff = abs(top_label[1] - second_label[1]) if second_label else float('inf')
    second_diff = abs(second_label[1] - third_label[1]) if third_label else float('inf')

    # Case 1: If the original USCS is a single label and ranks in the top 3, return the original USCS
    uscs_value = row['USCS']
    if isinstance(uscs_value, str) and uscs_value != "NOT APPLICABLE" and len(uscs_value.split('-')) == 1 and uscs_value in [top_label[0], second_label[0], third_label[0]]:
        predicted_label = uscs_value
    # Case 2: If the top 1 probability is significantly higher than the second, return the top 1 label
    elif top_label[1] > second_label[1] + threshold_1:
        predicted_label = top_label[0]
    # Case 3: If the top 1 and top 2 labels are very close (based on threshold_2), return a joint label
    elif top_diff < threshold_2:
        predicted_label = f"{top_label[0]}-{second_label[0]}"
    # Case 4: If all probabilities are extremely low or zero, return the original USCS label
    elif all(prob == 0 for _, prob in sorted_label_prob_pairs):
        predicted_label = uscs_value
    else:
        # Case 5: Default behavior, return the top label if none of the above cases apply
        predicted_label = top_label[0]

    return sorted_label_prob_pairs, predicted_label

# Make a copy of the layering dataframe to keep original intact
layering_score = layering.copy()

# Apply the function to generate 'USCS_prob' and 'USCS_predict'
layering_score[['USCS_prob', 'USCS_predict']] = layering_score.apply(
    lambda row: pd.Series(create_prob_list(row)), axis=1)

def create_prob_list(row):
    # Convert the row (which is a series) into a list of label-probability pairs
    label_prob_pairs = [(label, prob) for label, prob in row.items() if isinstance(prob, (int, float)) and prob > 0]

    # If no probabilities are above 0, return an empty list (for USCS_prob) and 'ANY' for USCS_predict
    if not label_prob_pairs:
        return [], 'ANY'

    # Sort the list by probability in increasing order
    sorted_label_prob_pairs = sorted(label_prob_pairs, key=lambda x: x[1])

    # Set the USCS_predict as the label with the highest probability
    predicted_label = sorted_label_prob_pairs[-1][0]

    return sorted_label_prob_pairs, predicted_label

# Apply the function to get both the sorted probabilities and the prediction
layering_score[['USCS_prob', 'USCS_predict']] = layering_score.apply(
    lambda row: pd.Series(create_prob_list(row)), axis=1)

layering_score


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Borehole ID,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology,USCS_prob,USCS_predict
0,1,1,0.0,1.2,"Duff and roots over loose, dark brown, silty f...",PT,Fill,"[(Borehole ID, 1), (Sub Borehole Layer, 1), (B...",Bottom Depth
1,1,2,1.2,2.7,"Soft, gray-brown, mottled, very fine, sandy si...",ML,Sandy silt,"[(Borehole ID, 1), (Top Depth, 1.200000048), (...",Bottom Depth
2,1,3,2.7,8.0,"Very stiff to hard, gray-brown, silt with some...",ML,Silt,"[(Borehole ID, 1), (Top Depth, 2.700000048), (...",Bottom Depth
3,2,1,0.0,0.8,"Duff, topsoil and roots, moist",PT,Topsoil / vegetation,"[(Bottom Depth, 0.800000012), (Sub Borehole La...",Borehole ID
4,2,2,0.8,2.5,"Loose, reddish-brown, silty sand, moist to wet",SM,Silty sand,"[(Top Depth, 0.800000012), (Borehole ID, 2), (...",Bottom Depth
...,...,...,...,...,...,...,...,...,...
341153,153209,3,25.0,28.5,"Clean silty sands, medium to dense to dense.",SM-SP,Silty sand,"[(Sub Borehole Layer, 3), (Top Depth, 25.0), (...",Borehole ID
341154,153210,1,0.0,6.0,"Silty to clayey sands and sandy silts, loose.",SM-ML,Clayey sand,"[(Sub Borehole Layer, 1), (Bottom Depth, 6.0),...",Borehole ID
341155,153210,2,6.0,19.0,"Fine sandy and clayey silts, very soft to medi...",ML,Clayey silt,"[(Sub Borehole Layer, 2), (Top Depth, 6.0), (B...",Borehole ID
341156,153210,3,19.0,30.0,"Silty sand, medium dense to dense.",SM-SP,Silty sand,"[(Sub Borehole Layer, 3), (Top Depth, 19.0), (...",Borehole ID


In [ ]:

# Now add the result back to the original 'layering' dataframe
layering['USCS_prob'] = layering_score['USCS_prob']
layering['USCS_predict'] = layering_score['USCS_predict']

layering

## SPTN to USCS approach 2

In [ ]:
boreholes_uscs_sptn

,Borehole ID,Latitude,Longitude,Elevation,Borehole Test ID,Test Depth,SPTN,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology,Density/Consistency
0,7,47.575005,-122.406037,131.30,1,98.30,60.0,8,119.80,94.80,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay,Hard
1,7,47.575005,-122.406037,131.30,2,95.80,58.0,8,119.80,94.80,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay,Hard
2,7,47.575005,-122.406037,131.30,3,93.30,80.0,11,93.30,90.30,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand,Hard
3,7,47.575005,-122.406037,131.30,4,90.80,84.0,11,93.30,90.30,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand,Hard
4,8,47.574982,-122.406239,130.90,1,97.40,94.0,6,99.40,94.90,"2 inches of moist, gray/green, clayey silty fi...",ML,Silty sand,Hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28927,153205,47.437412,-122.242456,23.75,9,-8.75,44.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense
28928,153205,47.437412,-122.242456,23.75,10,-13.75,23.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Medium
28929,153205,47.437412,-122.242456,23.75,11,-18.75,43.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense
28930,153205,47.437412,-122.242456,23.75,12,-23.75,27.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense


In [ ]:
layering

,Borehole ID,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology
0,1,1,0.0,1.2,"Duff and roots over loose, dark brown, silty f...",PT,Fill
1,1,2,1.2,2.7,"Soft, gray-brown, mottled, very fine, sandy si...",ML,Sandy silt
2,1,3,2.7,8.0,"Very stiff to hard, gray-brown, silt with some...",ML,Silt
3,2,1,0.0,0.8,"Duff, topsoil and roots, moist",PT,Topsoil / vegetation
4,2,2,0.8,2.5,"Loose, reddish-brown, silty sand, moist to wet",SM,Silty sand
...,...,...,...,...,...,...,...
341153,153209,3,25.0,28.5,"Clean silty sands, medium to dense to dense.",SM-SP,Silty sand
341154,153210,1,0.0,6.0,"Silty to clayey sands and sandy silts, loose.",SM-ML,Clayey sand
341155,153210,2,6.0,19.0,"Fine sandy and clayey silts, very soft to medi...",ML,Clayey silt
341156,153210,3,19.0,30.0,"Silty sand, medium dense to dense.",SM-SP,Silty sand


## Baseline
Linear model, on boreholes_uscs_sptn with no more preprocessing

In [ ]:
boreholes_uscs_sptn.head()

In [ ]:
boreholes_uscs_sptn_factorize = boreholes_uscs_sptn.copy()

# Factorize and correctly assign only the first returned value
boreholes_uscs_sptn_factorize['USCS_ID'] = pd.factorize(boreholes_uscs_sptn_factorize['USCS'])[0] + 1
boreholes_uscs_sptn_factorize['Lithology_ID'] = pd.factorize(boreholes_uscs_sptn_factorize['Lithology'])[0] + 1
boreholes_uscs_sptn_factorize['Description_ID']= pd.factorize(boreholes_uscs_sptn_factorize['Description'])[0] + 1

boreholes_uscs_sptn_factorize.drop(columns=['Borehole ID', 'Borehole Test ID', 'USCS', 'Lithology', 'Description'], inplace=True)
print(boreholes_uscs_sptn_factorize.columns)
boreholes_uscs_sptn_factorize

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, mean_absolute_percentage_error


boreholes_uscs_sptn_factorize.dropna(inplace=True)

X_cols = ['Latitude', 'Longitude', 'Elevation', 'Test Depth']
Y_cols = ['SPTN', 'Sub Borehole Layer', 'Top Depth', 'Bottom Depth', 'USCS_ID',
          'Lithology_ID', 'Description_ID']

X = boreholes_uscs_sptn_factorize[X_cols]
Y = boreholes_uscs_sptn_factorize[Y_cols]

# Train-test split (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, Y_train)

# Make predictions
Y_pred = model.predict(X_test)

mse = mean_squared_error(Y_test, Y_pred, multioutput='raw_values')  # Mean Squared Error
mae = mean_absolute_error(Y_test, Y_pred, multioutput='raw_values')  # Mean Absolute Error
mape = mean_absolute_percentage_error(Y_test, Y_pred)  # Mean Absolute Percentage Error
r2 = r2_score(Y_test, Y_pred, multioutput='variance_weighted')  # R² Score
explained_var = explained_variance_score(Y_test, Y_pred, multioutput='variance_weighted')  # Explained Variance Score

# Print results
print("Mean Squared Error (MSE) for each output:", mse)
print("Mean Absolute Error (MAE) for each output:", mae)
print("Mean Absolute Percentage Error (MAPE):", mape)
print("Overall R² Score:", r2)
print("Explained Variance Score:", explained_var)

## Predict

In [22]:
boreholes_uscs_sptn

,Borehole ID,Latitude,Longitude,Elevation,Borehole Test ID,Test Depth,SPTN,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology,Density/Consistency
0,7,47.575005,-122.406037,131.30,1,98.30,60.0,8,119.80,94.80,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay,Hard
1,7,47.575005,-122.406037,131.30,2,95.80,58.0,8,119.80,94.80,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay,Hard
2,7,47.575005,-122.406037,131.30,3,93.30,80.0,11,93.30,90.30,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand,Hard
3,7,47.575005,-122.406037,131.30,4,90.80,84.0,11,93.30,90.30,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand,Hard
4,8,47.574982,-122.406239,130.90,1,97.40,94.0,6,99.40,94.90,"2 inches of moist, gray/green, clayey silty fi...",ML,Silty sand,Hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28927,153205,47.437412,-122.242456,23.75,9,-8.75,44.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense
28928,153205,47.437412,-122.242456,23.75,10,-13.75,23.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Medium
28929,153205,47.437412,-122.242456,23.75,11,-18.75,43.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense
28930,153205,47.437412,-122.242456,23.75,12,-23.75,27.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense


In [23]:
boreholes_uscs_sptn.to_csv('boreholes_uscs_sptn.csv')

## Kriging

In [25]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Selecting relevant columns
features = ["Latitude", "Longitude", "Elevation"]
target_numeric = "SPTN"
target_categorical = "USCS"

df = boreholes_uscs_sptn
# Drop any rows with missing values
df_clean = df.dropna(subset=features + [target_numeric, target_categorical])

# Encode the categorical target (USCS) for classification
label_encoder = LabelEncoder()
df_clean["USCS_encoded"] = label_encoder.fit_transform(df_clean[target_categorical])

# Normalize the feature columns
scaler = StandardScaler()
df_clean[features] = scaler.fit_transform(df_clean[features])

# Display the processed dataset
df_clean


<ipython-input-25-b524c0cf5668>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["USCS_encoded"] = label_encoder.fit_transform(df_clean[target_categorical])
<ipython-input-25-b524c0cf5668>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[features] = scaler.fit_transform(df_clean[features])


,Borehole ID,Latitude,Longitude,Elevation,Borehole Test ID,Test Depth,SPTN,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology,Density/Consistency,USCS_encoded
0,7,-0.073215,-0.234137,-0.178915,1,98.30,60.0,8,119.80,94.80,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay,Hard,7
1,7,-0.073215,-0.234137,-0.178915,2,95.80,58.0,8,119.80,94.80,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay,Hard,7
2,7,-0.073215,-0.234137,-0.178915,3,93.30,80.0,11,93.30,90.30,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand,Hard,36
3,7,-0.073215,-0.234137,-0.178915,4,90.80,84.0,11,93.30,90.30,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand,Hard,36
4,8,-0.073284,-0.234354,-0.179919,1,97.40,94.0,6,99.40,94.90,"2 inches of moist, gray/green, clayey silty fi...",ML,Silty sand,Hard,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28927,153205,-0.495261,-0.057596,-0.448769,9,-8.75,44.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense,68
28928,153205,-0.495261,-0.057596,-0.448769,10,-13.75,23.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Medium,68
28929,153205,-0.495261,-0.057596,-0.448769,11,-18.75,43.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense,68
28930,153205,-0.495261,-0.057596,-0.448769,12,-23.75,27.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense,68


In [27]:
!pip install pykrige

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.6/979.6 kB 8.3 MB/s eta 0:00:00


In [28]:
from pykrige.rk import Krige
from pykrige import variogram_models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_clean[features], df_clean[target_numeric], test_size=0.2, random_state=42)

# Apply Kriging for SPTN prediction
kriging_model = Krige(model=variogram_models.spherical)
kriging_model.fit(X_train, y_train)
sptn_predictions = kriging_model.predict(X_test)

# Evaluate Kriging model performance
sptn_mae = np.mean(np.abs(sptn_predictions - y_test))
sptn_mae


AttributeError: module 'pykrige.variogram_models' has no attribute 'spherical'

## Random Forest

In [31]:
layering

,Borehole ID,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology
0,1,1,0.0,1.2,"Duff and roots over loose, dark brown, silty f...",PT,Fill
1,1,2,1.2,2.7,"Soft, gray-brown, mottled, very fine, sandy si...",ML,Sandy silt
2,1,3,2.7,8.0,"Very stiff to hard, gray-brown, silt with some...",ML,Silt
3,2,1,0.0,0.8,"Duff, topsoil and roots, moist",PT,Topsoil / vegetation
4,2,2,0.8,2.5,"Loose, reddish-brown, silty sand, moist to wet",SM,Silty sand
...,...,...,...,...,...,...,...
341153,153209,3,25.0,28.5,"Clean silty sands, medium to dense to dense.",SM-SP,Silty sand
341154,153210,1,0.0,6.0,"Silty to clayey sands and sandy silts, loose.",SM-ML,Clayey sand
341155,153210,2,6.0,19.0,"Fine sandy and clayey silts, very soft to medi...",ML,Clayey silt
341156,153210,3,19.0,30.0,"Silty sand, medium dense to dense.",SM-SP,Silty sand


In [32]:
df

,Borehole ID,Latitude,Longitude,Elevation,Borehole Test ID,Test Depth,SPTN,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology,Density/Consistency
0,7,47.575005,-122.406037,131.30,1,98.30,60.0,8,119.80,94.80,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay,Hard
1,7,47.575005,-122.406037,131.30,2,95.80,58.0,8,119.80,94.80,"Moist, gray, silty clay, very thin interbeds o...",CL-ML,Silty clay,Hard
2,7,47.575005,-122.406037,131.30,3,93.30,80.0,11,93.30,90.30,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand,Hard
3,7,47.575005,-122.406037,131.30,4,90.80,84.0,11,93.30,90.30,"Moist, gray/green, silty fine sand, moderately...",ML,Silty sand,Hard
4,8,47.574982,-122.406239,130.90,1,97.40,94.0,6,99.40,94.90,"2 inches of moist, gray/green, clayey silty fi...",ML,Silty sand,Hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28927,153205,47.437412,-122.242456,23.75,9,-8.75,44.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense
28928,153205,47.437412,-122.242456,23.75,10,-13.75,23.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Medium
28929,153205,47.437412,-122.242456,23.75,11,-18.75,43.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense
28930,153205,47.437412,-122.242456,23.75,12,-23.75,27.0,6,-3.25,-30.25,"Dark gray silty fine to medium sand, loose bec...",SM,Silty sand,Dense


In [29]:
# Drop rows with missing Latitude, Longitude, or Elevation
df_clean = df.dropna(subset=['Latitude', 'Longitude', 'Elevation'])

# Separate datasets for classification (USCS) and regression (SPTN)
df_uscs = df_clean.dropna(subset=['USCS'])  # Classification target
df_sptn = df_clean.dropna(subset=['SPTN'])  # Regression target

# Encode the USCS categorical target variable
df_uscs['USCS_encoded'] = df_uscs['USCS'].astype('category').cat.codes

# Display cleaned dataset info
df_uscs.info(), df_sptn.info()


<class 'pandas.core.frame.DataFrame'>
Index: 17427 entries, 0 to 28931
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Borehole ID          17427 non-null  int64  
 1   Latitude             17427 non-null  float64
 2   Longitude            17427 non-null  float64
 3   Elevation            17427 non-null  float64
 4   Borehole Test ID     17427 non-null  int64  
 5   Test Depth           17427 non-null  float64
 6   SPTN                 13180 non-null  float64
 7   Sub Borehole Layer   17427 non-null  int64  
 8   Top Depth            17427 non-null  float64
 9   Bottom Depth         17427 non-null  float64
 10  Description          17427 non-null  object 
 11  USCS                 17427 non-null  object 
 12  Lithology            17427 non-null  object 
 13  Density/Consistency  17145 non-null  object 
 14  USCS_encoded         17427 non-null  int8   
dtypes: float64(7), int64(3), int8(1), object(

<ipython-input-29-cb67bdf0bf34>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uscs['USCS_encoded'] = df_uscs['USCS'].astype('category').cat.codes


(None, None)

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error

# Define features and targets
features = ['Elevation', 'Latitude', 'Longitude']

# Prepare data for classification (USCS)
X_uscs = df_uscs[features]
y_uscs = df_uscs['USCS_encoded']
X_train_uscs, X_test_uscs, y_train_uscs, y_test_uscs = train_test_split(X_uscs, y_uscs, test_size=0.2, random_state=42)

# Prepare data for regression (SPTN)
X_sptn = df_sptn[features]
y_sptn = df_sptn['SPTN']
X_train_sptn, X_test_sptn, y_train_sptn, y_test_sptn = train_test_split(X_sptn, y_sptn, test_size=0.2, random_state=42)

# Train Random Forest Classifier for USCS
clf_uscs = RandomForestClassifier(n_estimators=100, random_state=42)
clf_uscs.fit(X_train_uscs, y_train_uscs)
y_pred_uscs = clf_uscs.predict(X_test_uscs)
accuracy_uscs = accuracy_score(y_test_uscs, y_pred_uscs)

# Train Random Forest Regressor for SPTN
reg_sptn = RandomForestRegressor(n_estimators=100, random_state=42)
reg_sptn.fit(X_train_sptn, y_train_sptn)
y_pred_sptn = reg_sptn.predict(X_test_sptn)
mae_sptn = mean_absolute_error(y_test_sptn, y_pred_sptn)

# Display performance metrics
accuracy_uscs, mae_sptn


(0.5017211703958692, 17.167418072468376)

## USCS

In [35]:
layering

,Borehole ID,Sub Borehole Layer,Top Depth,Bottom Depth,Description,USCS,Lithology
0,1,1,0.0,1.2,"Duff and roots over loose, dark brown, silty f...",PT,Fill
1,1,2,1.2,2.7,"Soft, gray-brown, mottled, very fine, sandy si...",ML,Sandy silt
2,1,3,2.7,8.0,"Very stiff to hard, gray-brown, silt with some...",ML,Silt
3,2,1,0.0,0.8,"Duff, topsoil and roots, moist",PT,Topsoil / vegetation
4,2,2,0.8,2.5,"Loose, reddish-brown, silty sand, moist to wet",SM,Silty sand
...,...,...,...,...,...,...,...
341153,153209,3,25.0,28.5,"Clean silty sands, medium to dense to dense.",SM-SP,Silty sand
341154,153210,1,0.0,6.0,"Silty to clayey sands and sandy silts, loose.",SM-ML,Clayey sand
341155,153210,2,6.0,19.0,"Fine sandy and clayey silts, very soft to medi...",ML,Clayey silt
341156,153210,3,19.0,30.0,"Silty sand, medium dense to dense.",SM-SP,Silty sand


In [34]:
# Drop rows with missing Latitude, Longitude, or Elevation
df = layering
df_clean = df.dropna(subset=['Latitude', 'Longitude', 'Elevation'])

# Separate datasets for classification (USCS) and regression (SPTN)
df_uscs = df_clean.dropna(subset=['USCS'])  # Classification target

# Encode the USCS categorical target variable
df_uscs['USCS_encoded'] = df_uscs['USCS'].astype('category').cat.codes

# Display cleaned dataset info
df_uscs.info(), df_sptn.info()


KeyError: ['Latitude', 'Longitude', 'Elevation']